# Batch Jobs with Gemini

This notebook demonstrates how to use the Gemini API's Batch Mode. This feature is designed to process large volumes of non-urgent requests asynchronously.

Key benefits of using Batch Mode include:
- **Cost-Effective**: Priced at 50% of the standard cost for the same model.
- **High Throughput**: Ideal for large-scale data processing, pre-processing, or evaluations.
- **Asynchronous**: Submit jobs and retrieve results later, with a target turnaround of 24 hours (though often much faster).

You can submit batch requests in two ways:
1.  **Inline Requests**: For smaller batches, directly including requests in the API call.
2.  **Input File**: For larger batches, using a JSON Lines (JSONL) file uploaded to the Google AI File API.

This guide will walk you through both methods.

In [ ]:
%uv pip install google-genai --upgrade

## Setup

Import the necessary libraries and configure your API key. Remember to set your `GEMINI_API_KEY` as an environment variable.

In [2]:
import google.genai as genai
from google.genai import types

client = genai.Client()

## 1. Inline Requests

For smaller batches, you can embed the requests directly in your call. This is convenient for a handful of prompts.

In [ ]:
# A list of dictionaries, where each is a GenerateContentRequest
inline_requests = [
    {
        'contents': [{
            'parts': [{'text': 'Tell me a one-sentence joke.'}],
            'role': 'user'
        }]
    },
    {
        'contents': [{
            'parts': [{'text': 'Why is the sky blue?'}],
            'role': 'user'
        }]
    }
]

# Create the batch job
inline_batch_job = client.batches.create(
    model="gemini-2.5-flash", # Or another supported model
    src=inline_requests,
    config={
        'display_name': "inlined-requests-job-1",
    },
)

print(f"Created batch job: {inline_batch_job.name}")

Created batch job: batches/xg0jgclsevahb3nb79ne9wt3xsldeqmjk63q


### Monitor the Job and Retrieve Results

After creating a job, you need to poll its status until it completes. A job can be `JOB_STATE_PENDING`, `JOB_STATE_SUCCEEDED`, `JOB_STATE_FAILED`, or `JOB_STATE_CANCELLED`.

In [48]:
import time
import json
import google.genai as genai

# Assume 'client' is an initialized genai.Client() instance
# client = genai.Client()

def monitor_and_get_batch_results(job_name: str, client: genai.client.Client,max_retries: int = 10):
    completed_states = {
        'JOB_STATE_SUCCEEDED',
        'JOB_STATE_FAILED',
        'JOB_STATE_CANCELLED',
    }

    print(f"Polling status for job: {job_name}")
    batch_job = client.batches.get(name=job_name)
    retry_count = 0
    while batch_job.state.name not in completed_states and retry_count < max_retries:
        print(f"Current state: {batch_job.state.name}")
        time.sleep(10)  # Wait for 10 seconds before polling again
        batch_job = client.batches.get(name=job_name)
        retry_count += 1
    if retry_count == max_retries:
        print("No results found after max retries. Try again later.")
        return None
        
    print(f"Job finished with state: {batch_job.state.name}")

    # Handle failed or cancelled jobs
    if batch_job.state.name in ('JOB_STATE_FAILED', 'JOB_STATE_CANCELLED'):
        if batch_job.error:
            print(f"Error: {batch_job.error}")
        else:
            print(f"Job was {batch_job.state.name.lower().split('_')[-1]}.")
        return None

    # Retrieve and return results for succeeded jobs
    if batch_job.state.name == 'JOB_STATE_SUCCEEDED':
        # Case 1: Results are returned inline
        if batch_job.dest and batch_job.dest.inlined_responses:
            results = []
            for inline_response in batch_job.dest.inlined_responses:
                if inline_response.response:
                    try:
                        results.append(inline_response.model_dump())
                    except AttributeError:
                        results.append(inline_response.model_dump())
                elif inline_response.error:
                    results.append(inline_response.model_dump())
            return results

        # Case 2: Results are in a file
        elif batch_job.dest and batch_job.dest.file_name:
            result_file_name = batch_job.dest.file_name
            print(f"Results are in file: {result_file_name}. Downloading...")            
            file_content_bytes = client.files.download(file=result_file_name)            
            # Parse the JSONL string into a list of dictionaries
            parsed_responses = [
                json.loads(line) for line in file_content_bytes.decode('utf-8').strip().split('\n')
            ]
            return parsed_responses
    
    # Return None if no results are found
    print("No results found (neither file nor inline).")
    return None


# Assuming you have a job name from a previously created job
inline_job_name = inline_batch_job.name # ,e.g. "batches/xg0jgclsevahb3nb79ne9wt3xsldeqmjk63q" # or "batches/xg0jgclsevahb3nb79ne9wt3xsldeqmjk63q"

# Call the function to get the results
inline_results = monitor_and_get_batch_results(inline_job_name, client)

for result in inline_results:
    print("-" * 20)
    parts = result['response']['candidates'][0]['content']['parts']
    for part in parts:
        print(part['text'])

Polling status for job: batches/xg0jgclsevahb3nb79ne9wt3xsldeqmjk63q
Job finished with state: JOB_STATE_SUCCEEDED
--------------------
Why don't scientists trust atoms? Because they make up everything!
--------------------
The reason the sky is blue is due to a phenomenon called **Rayleigh Scattering**. Here's a breakdown:

1.  **Sunlight is White Light:** Sunlight, which appears white to us, is actually made up of all the colors of the rainbow (the visible spectrum). Each color has a different wavelength: violet and blue have shorter, smaller wavelengths, while red and orange have longer, larger wavelengths.

2.  **Earth's Atmosphere:** Our planet is surrounded by an atmosphere primarily composed of tiny gas molecules, mainly nitrogen (N2) and oxygen (O2).

3.  **Scattering of Light:** When sunlight enters the atmosphere, it collides with these gas molecules. This causes the light to be scattered in all directions.

4.  **Rayleigh Scattering Explained:**
    *   **Size Matters:** The 

## 2. Input File for Large Batches

For many requests, the recommended approach is to use a JSON Lines (JSONL) file. Each line contains a user-defined `key` and a `request` object. This file is then uploaded using the File API.

In [7]:
# Create a sample JSONL file
with open("my-batch-requests.jsonl", "w") as f:
    requests = [
        {"key": "request-1", "request": {"contents": [{"parts": [{"text": "Describe the process of photosynthesis."}]}], "generation_config": {"temperature": 0.7}}},
        {"key": "request-2", "request": {"contents": [{"parts": [{"text": "What are the main ingredients in a Margherita pizza?"}]}]}}
    ]
    for req in requests:
        f.write(json.dumps(req) + "\n")

# Upload the file to the File API
uploaded_file = client.files.upload(
    file='my-batch-requests.jsonl',
    config=types.UploadFileConfig(display_name='my-batch-requests', mime_type='application/jsonl')
)

print(f"Uploaded file: {uploaded_file.name}")

Uploaded file: files/itwjg9sf26gk


In [9]:
# Create the batch job using the uploaded file
file_batch_job = client.batches.create(
    model="gemini-2.5-flash",
    src=uploaded_file.name,
)

print(f"Created batch job from file: {file_batch_job.name}")

Created batch job from file: batches/juxtg5cd5ow3xnkc2kck3wotk718t60itzph


### Monitor the Job and Retrieve File Results

The monitoring process is the same, but the results are returned in an output file instead of being inline.

In [49]:
# Assuming you have a job name from a previously created job
inline_job_name = file_batch_job.name # ,e.g. "batches/juxtg5cd5ow3xnkc2kck3wotk718t60itzph" # or "batches/xg0jgclsevahb3nb79ne9wt3xsldeqmjk63q"

# Call the function to get the results
file_results = monitor_and_get_batch_results(inline_job_name, client)

for result in file_results:
    print(result)
    print("-" * 20)
    parts = result['response']['candidates'][0]['content']['parts']
    for part in parts:
        print(part['text'])

Polling status for job: batches/juxtg5cd5ow3xnkc2kck3wotk718t60itzph
Job finished with state: JOB_STATE_SUCCEEDED
Results are in file: files/batch-juxtg5cd5ow3xnkc2kck3wotk718t60itzph. Downloading...
{'response': {'responseId': 'tMlraPrbHNOG-8YPyMuPwQ4', 'candidates': [{'content': {'role': 'model', 'parts': [{'text': "Photosynthesis is the fundamental biological process by which green plants, algae, and some bacteria convert light energy into chemical energy. This chemical energy is stored in organic compounds, primarily glucose (a sugar), which serves as the primary food source for the organism. In doing so, photosynthesis also releases oxygen as a byproduct, which is essential for most life on Earth.\n\nThe overall simplified equation for photosynthesis is:\n\n$6CO_2$ (Carbon Dioxide) + $6H_2O$ (Water) + Light Energy $\\rightarrow C_6H_{12}O_6$ (Glucose) + $6O_2$ (Oxygen)\n\nThis complex process occurs primarily in specialized organelles called **chloroplasts** within the cells of pl